# ImkLib2 Basic Usage

imklib2 is currently not available through remote maven repositories. To install into your local maven repository, run
```
./gradlew build publishToMavenLocal
```
in the project's root directory.

In [1]:
// set up dependencies

// use local repo for now; not deployed to remote maven repo yet
@file:Repository("*mavenLocal")
@file:Repository("https://maven.scijava.org/content/groups/public")
// requires installation into local maven repository (./gradlew build publishToMavenLocal)
@file:DependsOn("net.imglib2:imklib2:0.1.0-SNAPSHOT")
@file:DependsOn("net.imglib2:imglib2-label-multisets:0.8.1")
@file:DependsOn("sc.fiji:bigdataviewer-vistools:1.0.0-beta-21")
%use lets-plot

Simply import all names from the `net.imglib2.imklib` package to get started:

In [2]:
import net.imglib2.imklib.*

Other imports

In [3]:
import bdv.util.BdvFunctions
import bdv.util.BdvOptions
import kotlin.math.sqrt
import kotlin.random.Random
import net.imglib2.realtransform.AffineTransform2D
import net.imglib2.type.numeric.ARGBType

## Generate Data

Create ImgLib2 `ArrayImg`s with the `imgs` object. The images are backed by Java primitive type arrays. Images can be created
 1. with dimensions only (populate with default primitive type array value, usually `0`)
 2. with dimensions and initializer that consumes linear index and type at voxel
 3. with dimensions and initializer that produces a primitive type value for linear index

Note: Use `flatStringRepresentation` only for small images.

In [4]:
// dims only (1)
println(imklib.booleans(1L, 2L, 3L).flatStringRepresentation)
// dims with initializer (2)
println(imklib.argbs(1L, 2L, 3L) { i, argb -> argb.set((i+5) * 100) }.flatStringRepresentation)
// dims with initializer (3)
println(imklib.unsignedLongs(1L, 2L, 3L) { _ -> Long.MIN_VALUE }.flatStringRepresentation)
println(imklib.floats(1L, 2L, 3L) { i -> i.toFloat() + 0.5f }.flatStringRepresentation)
println(imklib.doubles(1L, 2L, 3L) { i -> i.toDouble() + 1.0 }.flatStringRepresentation)

Line_11.jupyter.kts (4:16 - 21) None of the following functions can be called with the arguments supplied: 
public final fun argbs(vararg dim: Long): ArrayImg<ARGBType!, IntArray!>! defined in net.imglib2.imklib.imklib
public final inline fun argbs(vararg dim: Long, init: (Int) -> Int): ArrayImg<ARGBType!, *> defined in net.imglib2.imklib.imklib

Virtual images can be created without any data allocation. The values are generated on request. If no interval is specified, the domain of the image will be unbounded

In [5]:
val dims = longArrayOf(1L, 2L, 3L)
println(imklib.zeros(3))
println(imklib.zeros(dims.interval).flatStringRepresentation)

net.imglib2.util.ConstantUtils$1@1d533ca6
IntervalView [(0, 0, 0) -- (0, 1, 2) = 1x2x3]: [0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0]


You can restrict the bounds of any image (bound or unbound) to any interval (of the same dimensionality) with the bracket operator `[]`. Note that, for bounded images, the new bounds are not checked for consistency with existing bounds.

In [6]:
println(imklib.ones(3))
println(imklib.ones(3)[dims.interval].flatStringRepresentation)
println(imklib.ones(dims.interval).flatStringRepresentation)
println(imklib.ones(longArrayOf(1, 1, 1).interval).flatStringRepresentation)

net.imglib2.util.ConstantUtils$1@66ac9dfa
IntervalView [(0, 0, 0) -- (0, 1, 2) = 1x2x3]: [1.0 ,1.0 ,1.0 ,1.0 ,1.0 ,1.0]
IntervalView [(0, 0, 0) -- (0, 1, 2) = 1x2x3]: [1.0 ,1.0 ,1.0 ,1.0 ,1.0 ,1.0]
IntervalView [(0, 0, 0) -- (0, 0, 0) = 1x1x1]: [1.0]


For the inverse operation, bounded images can be extended with various strategies. In this example, the extended images are bounded to larger bounds than the original image to demonstrate the extension. One-dimensional images are used because the visualization with flat Strings will be more meaningful.

In [7]:
val dims = longArrayOf(5)
val extended = longArrayOf(-3, 7).intervalMinMax
val img = imklib.doubles(*dims) { i -> 3.5 - abs(2.0 - i) }
println("Original image:             ${img.flatStringRepresentation}")
println("Extend with zeros:          ${img.extendZero()[extended].flatStringRepresentation}")
println("Extend with value:          ${img.extendValue(3.14)[extended].flatStringRepresentation}")
println("Extend with border value:   ${img.extendBorder()[extended].flatStringRepresentation}")
println("Extend mirror single:       ${img.extendMirrorSingle()[extended].flatStringRepresentation}")
println("Extend mirror double:       ${img.extendMirrorDouble()[extended].flatStringRepresentation}")

Original image:             ArrayImg [5]: [1.5 ,2.5 ,3.5 ,2.5 ,1.5]
Extend with zeros:          IntervalView [(-3) -- (7) = 11]: [0.0 ,0.0 ,0.0 ,1.5 ,2.5 ,3.5 ,2.5 ,1.5 ,0.0 ,0.0 ,0.0]
Extend with value:          IntervalView [(-3) -- (7) = 11]: [3.14 ,3.14 ,3.14 ,1.5 ,2.5 ,3.5 ,2.5 ,1.5 ,3.14 ,3.14 ,3.14]
Extend with border value:   IntervalView [(-3) -- (7) = 11]: [1.5 ,1.5 ,1.5 ,1.5 ,2.5 ,3.5 ,2.5 ,1.5 ,1.5 ,1.5 ,1.5]
Extend mirror single:       IntervalView [(-3) -- (7) = 11]: [2.5 ,3.5 ,2.5 ,1.5 ,2.5 ,3.5 ,2.5 ,1.5 ,2.5 ,3.5 ,2.5]
Extend mirror double:       IntervalView [(-3) -- (7) = 11]: [3.5 ,2.5 ,1.5 ,1.5 ,2.5 ,3.5 ,2.5 ,1.5 ,1.5 ,2.5 ,3.5]


## Materializing RandomAccessibleIntervals

Many of the `RandomAccessibleIntervals` are views that are evaluated on request for each voxel. The data is not materialized and generated with each access. For expensive operations that will be accessed more than once, it may be beneficial to materialize or cache the results. `imklib` provides to options:
 1. `materialize` the `RandomAccessibleInterval` into an `ArrayImg` that is backed by a primitive type array
 2. `cache` the `RandomAcecssibleInterval` into a `CachedCellImg` with various caching strategies.

In [8]:
val dims = longArrayOf(10)
val img = imklib.doubles(*dims) { i -> i.toDouble() }
val bellCurve = (-((img - 5.0) `**` 2.0) / 4.0).exp()
val sqrt = bellCurve.convert(bellCurve.type) { s, t -> println("square-rooting $s"); t.setReal(sqrt(s.realDouble)) }
sqrt.flatStringRepresentation
println()
sqrt.flatStringRepresentation
println()
sqrt.flatStringRepresentation
println()
println("Materializing")
val materialized = sqrt.materialize()
materialized.flatStringRepresentation
materialized.flatStringRepresentation
materialized.flatStringRepresentation
println()
println("Caching with block size 5")
val cached = sqrt.cache(5)
cached.flatStringRepresentation
println()
cached.flatStringRepresentation
cached.flatStringRepresentation


square-rooting 0.00193045413622771
square-rooting 0.018315638888734186
square-rooting 0.10539922456186435
square-rooting 0.36787944117144233
square-rooting 0.7788007830714049
square-rooting 1.0
square-rooting 0.7788007830714049
square-rooting 0.36787944117144233
square-rooting 0.10539922456186435
square-rooting 0.018315638888734186

square-rooting 0.00193045413622771
square-rooting 0.018315638888734186
square-rooting 0.10539922456186435
square-rooting 0.36787944117144233
square-rooting 0.7788007830714049
square-rooting 1.0
square-rooting 0.7788007830714049
square-rooting 0.36787944117144233
square-rooting 0.10539922456186435
square-rooting 0.018315638888734186

square-rooting 0.00193045413622771
square-rooting 0.018315638888734186
square-rooting 0.10539922456186435
square-rooting 0.36787944117144233
square-rooting 0.7788007830714049
square-rooting 1.0
square-rooting 0.7788007830714049
square-rooting 0.36787944117144233
square-rooting 0.10539922456186435
square-rooting 0.018315638888734

CachedCellImg [10]: [0.04393693362340743 ,0.1353352832366127 ,0.32465246735834974 ,0.6065306597126334 ,0.8824969025845955 ,1.0 ,0.8824969025845955 ,0.6065306597126334 ,0.32465246735834974 ,0.1353352832366127]

## Arithmetic operations
`RandomAccessible`s and `RandomAccessibleInterval`s support arithmetic operations with other `RandomAccessible`s and `RandomAccessibleInterval`s and scalar values. Results of arithmetic operations are lazy views that are evaluated on request for each voxel. For expensive arithmetic operations, it may be beneficial to materialize these views into and `Img` if they are accessed multiple times.

In [9]:
val dims = longArrayOf(5)
val img = imklib.doubles(*dims) { i -> i + 1.0 }
val extended = img.extendBorder()
println("img: ${img.flatStringRepresentation}")
println("img squared: ${(img `**` 2.0).flatStringRepresentation}")
println("img + img: ${(img + img).flatStringRepresentation}")
println("gradient: ${(extended.translate(+1L) - extended.translate(-1L))[img].flatStringRepresentation}")
println("bell curve: ${(-((img - 3.1) `**` 2.0) / 6.25).exp().flatStringRepresentation}") 

img: ArrayImg [5]: [1.0 ,2.0 ,3.0 ,4.0 ,5.0]
img squared: net.imglib2.converter.read.ConvertedRandomAccessibleInterval@6e01b77f: [1.0 ,4.0 ,9.0 ,16.0 ,25.0]
img + img: net.imglib2.converter.read.BiConvertedRandomAccessibleInterval@2753122a: [2.0 ,4.0 ,6.0 ,8.0 ,10.0]
gradient: IntervalView [(0) -- (4) = 5]: [-1.0 ,-2.0 ,-2.0 ,-2.0 ,-1.0]
bell curve: net.imglib2.converter.read.ConvertedRandomAccessibleInterval@c855014: [0.49381219803334614 ,0.8239874333317032 ,0.9984012793176064 ,0.8784467393499313 ,0.5612437364432349]


In [10]:
val dims = longArrayOf(1000)
val img = imklib.doubles(*dims) { it.toDouble() }
println((img - 500.0).type::class)
val bellCurve = (-((img - 500.0) `**` 2.0) / 40000.0).exp()
val d = mapOf<String, Any>(
    "X" to img.map { it.realDouble },
    "bell curve" to bellCurve.flatIterable.map { it.realDouble }
)
val p = lets_plot(d)
p + geom_line { x = "X"; y = "bell curve" } + ggsize(800, 500)

class net.imglib2.type.numeric.real.DoubleType


## Coordinate Transformations

In [11]:
val rng = Random(100L)
val img = imklib.doubles(200L, 300L) { idx -> idx.toDouble() + 100 * rng.nextDouble() }
val rotation = AffineTransform2D().also {
    it.translate(-100.0, -150.0)
    it.rotate(-3 * PI / 4.0)
    it.translate(100.0, 150.0)
}
val transformedBoundingBox = img.transformBoundingBox(rotation).smallestsContaining
val rotated = img.extendZero().interpolatedNLinear.transform(rotation).rastered[transformedBoundingBox]
val bdv = BdvFunctions.show(img, "img", BdvOptions.options().is2D())
BdvFunctions.show(rotated, "rotated", BdvOptions.options().is2D().addTo(bdv))
null

[0.0, 0.0] [64.64466094067261, 326.7766952966369]
[199.0, 0.0] [-76.06958851545035, 186.06244584051393]
[0.0, 299.0] [276.06958851545033, 115.3517677218592]
[199.0, 299.0] [135.35533905932738, -25.36248173426378]


In [12]:
val url = "/home/zottel/Pictures/Screenshot_20200822_102323.png"
val img = imklib.io.open(url).asInts().asARGBs()
BdvFunctions.show(img, "screenshot")
null

In [13]:
val path = "/home/zottel/Downloads/sample_A_20160501.hdf"
val img = imklib.io.n5.openUntypedHDF5(path, "volumes/raw")
BdvFunctions.show(img.volatileView, "raw")
null

In [14]:
val path = "/home/zottel/Downloads/t1-head-1.tif"
val img = imklib.io.open(path)
BdvFunctions.show(img, "head")
null